In [1]:
# Tutorial: https://youtu.be/rc_Y6rdBqXM
#!pip install binance
!pip install python-binance


[notice] A new release of pip available: 22.3.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
# Binance, including sockets
import asyncio
from binance import BinanceSocketManager
from binance.client import Client
from binance import Client#, ThreadedWebsocketManager, ThreadedDepthCacheManager

from secret_folder import keys_testnet

# Standart libraries
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objs as go

import time

# API keys:
api_key = keys_testnet.api_key
api_secret = keys_testnet.api_secret
client = Client(api_key, api_secret, testnet=True)
client.API_URL = 'https://testnet.binance.vision/api'

# Settings:
symb = "BTCUSDT"
interval = "5MINUTE"

#candles = f"client.get_klines(symbol=symb, interval=Client.KLINE_INTERVAL_{interval})"

In [3]:
candles = client.get_klines(symbol=symb, interval=Client.KLINE_INTERVAL_5MINUTE)
numpycandles = np.array(candles)

# reshape date to pandas
df = pd.DataFrame(numpycandles.reshape(-1, 12), dtype=float, columns=('Open Time','Open','High','Low','Close','Volume','Close time','Quote asset volume','Number of trades','Taker buy base asset volume','Taker buy quote asset volume','Ignore'))
df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')

close_prices = df['Close']

# plot candlesticks using plotly
#fig = go.Figure(data=[go.Candlestick(x=df['Open Time'],
#                open=df['Open'],
#                high=df['High'],
#                low=df['Low'],
#                close=df['Close'])])

#fig.show()

In [4]:
# Define the RSI indicator
def rsi(price, period=14):
    delta = price.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(period).mean()
    avg_loss = loss.rolling(period).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

# Calculate the RSI indicator
rsi_indicator = rsi(df['Close'])


# Define Divergence 
def divergence(price_set, rsi_window=14, fast_ma = 1, slow_ma = 50):

# Calculate the RSI indicator for using in calculation of divergence 
   rsi_indicator = rsi(price_set)

# Calculate the Divergence indicator. 
   bullish_divergence = ((price_set.diff()<0) & (rsi_indicator.diff() > 0))
   bearish_divergence = ((price_set.diff()<0) & (rsi_indicator.diff() < 0))

   fast_ma_df = df[f'ma{fast_ma}'] = df['Close'].rolling(window=fast_ma).mean()
   slow_ma_df = df[f'ma{slow_ma}'] = df['Close'].rolling(window=slow_ma).mean()
   trend_up = fast_ma_df > slow_ma_df
   trend_down = fast_ma_df < slow_ma_df
   
   # Combine all previous indicators into the one 
   # We need that if bullish_divergence (trend_up), True PLUS fast MA above slow MA, then set 1, everything else - 0. If bearish_divergence True, PLUS fast MA above slow MA (trend_down), then set -1
   indicator = np.where( ( ( (bullish_divergence) == True)  & (trend_up == True) ), 1, 0)
   indicator = np.where( ( ( (bearish_divergence) == True)  & (trend_down == True) ), -1, indicator)
   
   return indicator

diver = divergence(close_prices, rsi_window=14, fast_ma = 1, slow_ma = 50)
last_indicator_value = diver[-1]

In [8]:
last_indicator_value

0

In [25]:
def trade(ticker, lotsize, open_position = False):
    try:
        if last_indicator_value > 0 and last_indicator_value != 0 and not open_position:
            order = client.create_order(symbol=ticker,
                                    side='BUY',
                                    type='MARKET',
                                    quantity=lotsize)
        print(order)
        buyprice = float(order['fills'][0]['price'])
        open_position = True
    except:
        time.sleep(61)  # sleep 61 seconds, if error
        

    while open_position:
        try:
            if open_position and last_indicator_value < 0 and last_indicator_value != 0:
                order = client.create_order(symbol=ticker,
                                        side='SELL',
                                        type='MARKET',
                                        quantity=lotsize)
        print(order)
        sellprice = float(order['fills'][0]['price'])
        open_position = False
    except:
        time.sleep(61)  # sleep 61 seconds, if error



SyntaxError: expected 'except' or 'finally' block (3083389836.py, line 22)

In [6]:
async def main(ticker, lotsize, open_position = False):
  bm = BinanceSocketManager(client)
  trade_socket = bm.trade_socket(ticker)
  async with trade_socket as tscm:
    while True:
      res = await tscm.recv()
      if last_indicator_value > 0 and last_indicator_value != 0 and not open_position:
        order = client.create_order(symbol=ticker,
                                    side='BUY',
                                    type='MARKET',
                                    quantity=lotsize)
        print(order)
        buyprice = float(order['fills'][0]['price'])
        open_position = True
      if open_position and last_indicator_value < 0 and last_indicator_value != 0:
         order = client.create_order(symbol=ticker,
                                    side='SELL',
                                    type='MARKET',
                                    quantity=lotsize)
         print(order)
         loop.stop()

In [26]:
if __name__ == "__main__":
  loop = asyncio.get_event_loop()
  loop.run_until_complete(main('BTCUSDT', 0.0001))

RuntimeError: This event loop is already running

In [ ]:
"""1. Check balance
2. Check if price of current candle is not differ by x% from price from other exchange
3. Check if Ta-Lib indicator value not differ more than y% from another exchange
4. Check open positions
5. Check id position already open
6. Open position characteristics: long/short, timeframe, symbol, indicators values. Are any other positions with same characteristics? If yes, than it's duplicate 
7. Before Closing: check position
8. After closing: position closed? Ok, go check if we have opposite signal. We have opposite signal? Ok, go open opposite Trade Position
9. Position open time
10. How many bars passed
11. Get order ID
"""


# Define the signals
signals = pd.Series(index=price.index, dtype=bool)
signals.loc['2022-01-17'] = True  # Enter long position on 2022-01-17

# Define the position size
position_size = 1

# Define the number of bars to hold the position
bars_to_hold = 3

# Define a variable to keep track of the number of bars the position has been held
bars_held = 0

# Define the portfolio
portfolio = pd.DataFrame(index=price.index, columns=['position', 'position_value', 'cash'])

# Iterate over the trading session
for i in range(len(price)):
    # Check if the position is open
    if signals.iloc[i]:
        # Open the position
        portfolio.at[price.index[i], 'position'] = position_size
        portfolio.at[price.index[i], 'position_value'] = position_size * price.iloc[i]
        portfolio.at[price.index[i], 'cash'] = 0
        # Reset the number of bars the position has been held
        bars_held = 0
    elif pd.notna(portfolio.at[price.index[i], 'position']):
        # Increment the number of bars the position has been held
        bars_held += 1
        # Check if the position should be closed
        if bars_held >= bars_to_hold:
            # Close the position
            portfolio.at[price.index[i], 'position'] = 0
            portfolio.at[price.index[i], 'position_value'] = 0
            portfolio.at[price.index[i], 'cash'] = position_size * price.iloc[i]
            # Reset the number of bars the position has been held
            bars_held = 0
    else:
        # Keep the portfolio unchanged
        portfolio.iloc[i] = portfolio.iloc[i - 1]

# Calculate the total portfolio value
portfolio['total_value'] = portfolio['position_value'] + portfolio['cash']



------



# Download price data for AAPL
price = yf.download('AAPL', start='2015-01-01')

# Define the number of bars to hold the position
n_bars = 5

# Define the position size and initial capital
position_size = 1
initial_capital = 10000

# Create a DataFrame of position weights
position_weights = pd.DataFrame(index=price.index, columns=price.columns, data=0.0)
for i in range(len(price) - n_bars):
    position_weights.iloc[i + n_bars] = position_size

# Define the portfolio
portfolio = vbt.Portfolio.from_orders(price, position_weights, init_cash=initial_capital)

# Get the total profit
total_profit = portfolio.total_profit()

# Calculate the total returns
total_returns = (total_profit / initial_capital) + 1

print(f"Total profit: {total_profit:.2f}")
print(f"Total returns: {total_returns:.2f}")

In [ ]:
import datetime
import logging
import os
import time
from binance.client import Client
from binance.enums import *
from binance.exceptions import BinanceAPIException, BinanceOrderException
from db2 import Database
import pandas as pd
import requests

# https://binance-docs.github.io/apidocs/futures/en/#new-order-trade - описание параметров

# binance
api_key = "viP3n58T3Nh3YXEb8CYHCqGY9ESuiOiRjuLnUncaL24bw3R3dimgiU4ShZdnAc0G"
api_secret = "qXvPbJuGmDPjpTq6U5Aq3I3ewGODD0juZjEGV5yg4Md2SIziqtoQtpidzYLtp4lB"
client = Client(api_key, api_secret, testnet=True)
client.API_URL = 'https://testnet.binancefuture.com'
# telegram
bot_token = '5745010665:AAGKbvySZrPk1zmhv-behSqsj5VJuJAiNxI'
TELEGRAM_CHAT_ID = "-1001563615894"

# получить сводную информацию по бирже
# info = client.get_exchange_info()
# print(info)

test_coin_list = ['BNBUSDT', 'BTCUSDT',
    'ETHUSDT', 'LTCUSDT', 'TRXUSDT', 'XRPUSDT']

def send_tlg_message(data):
    try:
        send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + TELEGRAM_CHAT_ID + \
                    '&text=' + data
        response = requests.get(send_text)
        print("TELEGRAM:", data)
        return response.json()
    except Exception as e:
        print("[X] Telegram Error:\n>", e)
        logging.error(f"[X] Telegram Error:{e}")

def get_order_reverse_side(side):
    if side == 'BUY':
        return 'SELL'
    elif side == 'SELL':
        return 'BUY'
    else:
        print(f'Unknown parameter: get_order_reverse_side("{side}"):')
        return None

def main():

    # основной цикл
    while True:
        try:
            # проверить статусы всех активных ордеров
            # делаем запрос к БД
            with Database() as db:
                # db.create_tables()
                new_orders = db.get_data(f"SELECT * from orders")

                df_orders = pd.DataFrame(new_orders, columns=['id',
                                                              'signal_id',
                                                              'transact_time',
                                                              'symbol',
                                                              'price',
                                                              'origQty',
                                                              'side',
                                                              'state',
                                                              'tp',
                                                              'sl',
                                                              'tp_id',
                                                              'sl_id'])



                error_orders = df_orders[df_orders['state'].isin(['ERROR'])]
                for idx, row in error_orders.iterrows():
                    symbol = row['symbol']
                    order_id = row['id']
                    side = get_order_reverse_side(row['side'])
                    qty = row['origQty']
                    try:
                        order_state = client.get_order(
                            symbol=symbol,
                            orderId=order_id)
                    except BinanceAPIException as e:
                        if 'Order does not exist' in e.message:
                            db.execute(f"""UPDATE orders SET state='CANCELED' where id={order_id}""")
                            continue
                        else:
                            print(e)
                        continue
                    if order_state['status'] == 'CANCELED':
                        # ордер был отменен - помечаем в БД новое сосотояние ордера
                        db.execute(
                            f"""UPDATE orders SET state='CANCELED' where id={order_id}""")
                    elif order_state['status'] == 'FILLED':
                        # отменяем ошибочный ордер
                        try:
                            client.create_test_order(symbol=symbol, side=side, type='MARKET', quantity=qty,
                                                reduceOnly='true')
                            client.create_order(symbol=symbol, side=side, type='MARKET', quantity=qty, reduceOnly='true')
                            db.execute(f"""UPDATE orders SET state='CANCELED' where id={order_id}""")

                        except Exception as e:
                            print(e)
                            logging.error(f"Cancel order error (id={order_id}):", e)
                            send_tlg_message(f"Cancel order error (id={order_id}): {e}.")
                            continue
                        try:
                            if cancel['status'] == 'CANCELED':
                                db.execute(f"""UPDATE orders SET state='CANCELED BY TIME' where id={order_id}""")
                                logging.info(f"Order canceled by time expiration (id={order_id}).")
                                send_tlg_message(f"Order canceled by time expiration (id={order_id}).")
                        except Exception as e:
                            print(e)
                            logging.error(f"Update BD error (id={order_id}):", e)
                            continue


                new_orders = df_orders[df_orders['state'].isin(['NEW'])]

                for idx, row in new_orders.iterrows():
                    print(idx, row['id'], row['symbol'])
                    symbol = row['symbol']
                    order_id = row['id']
                    order_state = client.get_order(
                        symbol=symbol,
                        orderId=order_id)
                    print(order_state)
                    if order_state['status'] == 'CANCELED':
                        # ордер был отменен - помечаем в БД новое сосотояние ордера
                        db.execute(
                            f"""UPDATE orders SET state='CANCELED' where id={order_id}""")
                        send_tlg_message(f"UPDATE orders SET state='CANCELED' where id={order_id}")
                    elif order_state['status'] == 'FILLED':
                        # ордер был исполнен => выставляем tp and sl
                        # в БД прописываем статус заявки 'FILLED'
                        # выставляем тейк-профит
                        print('take profit=', row['tp'])
                        print('row["origQty"]', row['origQty'])
                        print('row["origQty"]', row['origQty'])
                        print("row['side']", row['side'])
                        print("get_order_reverse_side(row['side'])", get_order_reverse_side(row['side']))

                        try:
                            tp_order = client.create_order(
                                symbol=symbol,
                                side=get_order_reverse_side(row['side']),
                                type='TAKE_PROFIT_LIMIT',
                                quantity=row['origQty'],
                                price=row['tp'],
                                stopPrice=row['tp'],
                                timeInForce='GTC'
                            )
                            logging.info(f"TP is placed(id={order_id}, {symbol},"
                                         f" {get_order_reverse_side(row['side'])}, {row['tp']}):")
                            send_tlg_message(f"TP is placed(id={order_id}, {symbol},"
                                             f"{get_order_reverse_side(row['side'])}, {row['tp']}):")
                        except Exception as e:
                            logging.error(f"Take profit setup error (id={order_id}):", e)
                            send_tlg_message(f"Take profit setup error (id={order_id}):{e}")
                            db.execute(f"""UPDATE orders SET state='{e}' where id={order_id}""")
                            continue
                        try:
                            db.execute(f"""UPDATE orders SET state='TP placed' where id={order_id}""")
                            logging.info(f"""TP placed. id={order_id}""")
                            send_tlg_message(f"""TP placed. id={order_id}""")
                        except Exception as e:
                            print(e)
                            logging.error(f"Update BD error (id={order_id}):", e)
                            send_tlg_message(f"Update BD error (id={order_id}):{e}")
                            continue

                        # выставляем стоп-лосс
                        try:
                            sl_order = client.create_order(
                                symbol=symbol,
                                side=get_order_reverse_side(row['side']),
                                type='STOP_LOSS_LIMIT',
                                quantity=row['origQty'],
                                price=row['tp'],
                                stopPrice=row['tp'],
                                timeInForce='GTC'
                            )
                            logging.info(f"SL is placed(id={order_id}, {symbol},"
                                         f" {get_order_reverse_side(row['side'])}, {row['tp']}):", e)
                            send_tlg_message(f"SL is placed(id={order_id}, {symbol},"
                                         f" {get_order_reverse_side(row['side'])}, {row['tp']}):", e)
                        except Exception as e:
                            logging.error(f"Stoploss setup error (id={order_id}):", e)
                            db.execute(f"""UPDATE orders SET state='{e}' where id={order_id}""")
                            send_tlg_message(f"Stoploss setup error (id={order_id}): {e}")
                            continue

                        try:
                            db.execute(f"""UPDATE orders SET state='SL placed' where id={order_id}""")
                            logging.info(f"""SL placed. id={order_id}""")
                        except Exception as e:
                            print(e)
                            logging.error(f"Update BD error (id={order_id}):", e)
                            continue
                    else:
                        # проверка на срок жизни ордера и его отмена , если он уже не актуален (20 сут.)
                        if time.mktime((datetime.datetime.now() - datetime.timedelta(minutes=10)).timetuple()) * 1000 > \
                                order_state['time']:
                            print(f"Order live time expired. Orders [id={order_id}] must be cancelled")
                            try:
                                cancel = client.cancel_order(symbol=order_state['symbol'],
                                                             orderId=order_state['orderId'])
                            except Exception as e:
                                print(e)
                                logging.error(f"Cancel order error (id={order_id}):", e)
                                send_tlg_message(f"Cancel order error (id={order_id}): {e}.")
                                continue
                            try:
                                if cancel['status'] == 'CANCELED':
                                    db.execute(f"""UPDATE orders SET state='CANCELED BY TIME' where id={order_id}""")
                                    logging.info(f"Order canceled by time expiration (id={order_id}).")
                                    send_tlg_message(f"Order canceled by time expiration (id={order_id}).")
                            except Exception as e:
                                print(e)
                                logging.error(f"Update BD error (id={order_id}):", e)
                                continue

                # break
                print('Pause 2 sec...')
                print('*' * 30)
                time.sleep(2)
        except Exception as e:
            print("Exception:", e)
            break

if __name__ == "__main__":
    main()
    exit(0)
